In [74]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(100_000_000_000)

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca

import shared_utils
from utils import *

from siuba import *
import pandas as pd
import geopandas as gpd
import shapely

import datetime as dt
import time
from zoneinfo import ZoneInfo

import rt_analysis as rt
import importlib

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

In [83]:
importlib.reload(rt)

<module 'rt_analysis' from '/home/jovyan/data-analyses/rt_delay/rt_analysis.py'>

In [76]:
mts_itp_id = 278

In [77]:
analysis_date = dt.date(2022, 2, 16)

In [78]:
def get_vehicle_positions(itp_id, analysis_date):
    ''' 
    itp_id: an itp_id (string or integer)
    analysis_date: datetime.date
    
    Interim function for getting complete vehicle positions data for a single operator on a single date of interest.
    To be replaced as RT views are implemented...
    
    Currently drops positions for day after analysis date after 2AM, temporary fix to balance capturing trips crossing
    midnight with avoiding duplicates...
    '''

    
    next_date = analysis_date + dt.timedelta(days = 1)
    date_str = analysis_date.strftime('%Y-%m-%d')
    
    start = dt.datetime.combine(analysis_date, dt.time(0))
    start_ts = int(start.timestamp())
    end = start + dt.timedelta(days = 1, seconds = 2 * 60**2)
    end_ts = int(end.timestamp())
    
    filename = f'vp_{itp_id}_{date_str}.parquet'
    path = check_cached(filename)
    if path:
        print('found parquet')
        return pd.read_parquet(path)
    else:
        df = query_sql(f"""
        SELECT calitp_itp_id, calitp_url_number,
        header.timestamp AS header_timestamp, vehicle.timestamp AS vehicle_timestamp,
        vehicle.vehicle.label AS entity_id, vehicle.vehicle.id AS vehicle_id,
        vehicle.trip.tripId AS trip_id, vehicle.position.longitude AS vehicle_longitude,
        vehicle.position.latitude AS vehicle_latitude
        FROM `cal-itp-data-infra.gtfs_rt.vehicle_positions`
        WHERE calitp_itp_id = {itp_id} AND vehicle.timestamp > {start_ts} AND vehicle.timestamp < {end_ts}
        """)
        
        df = df >> distinct(_.vehicle_trip_id, _.vehicle_timestamp, _keep_all=True)
        df = df.dropna(subset=['vehicle_timestamp'])
        assert not df.empty, f'no vehicle positions data found for {date_str}'
        df.vehicle_timestamp = df.vehicle_timestamp.apply(convert_ts)
        df.header_timestamp = df.header_timestamp.apply(convert_ts)

        # assert df.vehicle_timestamp.min() < dt.datetime.combine(analysis_date, dt.time(0)), 'rt data starts after analysis date'
        # assert dt.datetime.combine(analysis_date, dt.time(hour=23, minute=59)) < df.vehicle_timestamp.max(), 'rt data ends early on analysis date'
        # if not df.vehicle_timestamp.min() < dt.datetime.combine(analysis_date, dt.time(0)):
        #     warnings.warn('rt data starts after analysis date')
        # if not dt.datetime.combine(end) < df.vehicle_timestamp.max():
        #     warnings.warn('rt data ends early on analysis date')

        df.to_parquet(f'{GCS_FILE_PATH}cached_views/{filename}')
        return df

In [79]:
pbar = tqdm()

0it [00:00, ?it/s]

In [84]:
mts = rt.OperatorDayAnalysis(mts_itp_id, analysis_date, pbar)

found parquet
found parquet
found parquet
found parquet
found_parquet
3 trips out of 8038 have no shape, droppingvehicle positions gdf must not be empty
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1

In [ ]:
# mts.set_filter(start_time='15:00', end_time='19:00')

In [88]:
m = mts.segment_speed_map(size = [1300, 700])

negative speed for shape 834_8_20, dropping
negative speed for shape 4_2_49, dropping
speed above 80 for shape 928_0_58, dropping
negative speed for shape 928_0_58, dropping
negative speed for shape 967_9_51, dropping
negative speed for shape 705_2_35, dropping
negative speed for shape 894_3_31, dropping
stop_speeds shape: (0, 17), shape_id: 894_3_32, direction_id: 1
local variable 'grp_levels' referenced before assignment
negative speed for shape 992_9_52, dropping
negative speed for shape 894_2_27, dropping
negative speed for shape 985_9_8, dropping
speed above 80 for shape 985_9_9, dropping
negative speed for shape 985_9_10, dropping
negative speed for shape 115_2_81, dropping
negative speed for shape 4_3_54, dropping
negative speed for shape 973_9_15, dropping
speed above 80 for shape 510_1_326, dropping
speed above 80 for shape 510_0_324, dropping
negative speed for shape 705_3_34, dropping
negative speed for shape 972_8_14, dropping
stop_speeds shape: (0, 17), shape_id: 950_2_25,

/opt/conda/lib/python3.9/site-packages/shapely/ops.py:634: ShapelyDeprecationWarning: GeometryTypeError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
/home/jovyan/data-analyses/rt_delay/rt_analysis.py:520: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.



In [89]:
m.save(f'./speedmaps/{278}_02_16_*midday*.html')